# Cleaning data and creating custom features for Give Me Some Credit Kaggle Challenge

1. Cleaning the monthly income from NA values
2. Clean the debt ratio by replacing NA by the mean (Should ideally be done after splitting and the mean should be calculated on the training set if you want to do an evaluation)
3. Creating a montlhy debt feature
    * monthly income multiplied by debt ratio if income is not 0
    * debt ratio if income is 0
4. Create a Balanced Income feature that take into account Income and debt ratio
    * Set income to 0 when negative.
5. Clean the number of dependents feature
    * set NA to zero
6. Create a Blanced Income per household members feature
7. Cleaning the Number of Times Late feature
    * Create a custom categorical feature that contains 2 different tags for each row that contains a Number of time late of either 96 or 98
    * Remove the 96 and 98 values (Replacing those values by NA or some other justifiable value)
8. Add a feature that compute the weighted sum of the number of time late per duration
    * weight of 3 for 90 days and more
    * weight of 2 for 60 to 89 days
    * weight of 1 for 30 to 59 days


In [3]:
from pandas import DataFrame, read_csv, to_numeric
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import pickle
from bigml.api import BigML
import kaggle

%matplotlib inline

In [4]:
compression = True  #Do we compress the csv files?

version='v1.3'

In [5]:
suffix = version + '.csv'
if compression:
    suffix +='.bz2'

filename_fulltrain = 'gmsc-fulltrain-' + suffix
filename_train = 'gmsc-train80-' + suffix
filename_valid = 'gmsc-valid20-' + suffix
filename_test = 'gmsc-test-' + suffix

filename_bigml_ids = 'bigml-' + suffix

## Loading csv files as data frames

Files must be placed in the same directory as this file. Alternatively, modify the relative path to those files.

In [6]:
!pwd
!ls
fulltrain=read_csv('./cs-training.csv',index_col=0)
test=read_csv('./cs-test.csv',index_col=0)

/home/devel/handson-ml2/ML-notebooks/GiveMeSomeCredit
README.md			 gmsc-test.csv.bz2
bigml-v1.3.csv.bz2		 gmsc-train80-v1.2-dev.csv.bz2
cs-test.csv			 gmsc-train80-v1.3.csv.bz2
cs-training.csv			 gmsc-train80.csv.bz2
custom_features.ipynb		 gmsc-valid-pred-v1.3.csv
evaluation.ipynb		 gmsc-valid-predv-1.3.csv.bz2
features.ipynb			 gmsc-valid20-v1.2-dev.csv.bz2
gmsc-fulltrain-v-excel.csv.bz2	 gmsc-valid20-v1.3.csv.bz2
gmsc-fulltrain-v-excel2.csv.bz2  gmsc-valid20.csv.bz2
gmsc-fulltrain-v1.2.csv.bz2	 gsmc_predictions-v-excel.csv
gmsc-fulltrain-v1.3.csv.bz2	 gsmc_predictions-v-excel2.csv
gmsc-fulltrain.csv		 gsmc_predictions-v1.3.csv
gmsc-fulltrain.csv.bz2		 gsmc_predictionsv1.2-dev.csv
gmsc-test-v-excel.csv.bz2	 gsmc_predictionsv1.2.csv
gmsc-test-v-excel2.csv.bz2	 kaggleprediction.csv
gmsc-test-v1.3.csv.bz2		 kaggleprediction.csv.bz2
gmsc-test.csv			 submit.ipynb
gmsc-test.csv-v-excel.csv.bz2	 training.ipynb
gmsc-test.csv-v1.2.csv.bz2


## Reordering the objective field column to the end

In [7]:
fulltrain=fulltrain[[c for c in fulltrain if c != 'SeriousDlqin2yrs']+['SeriousDlqin2yrs']]
test=test[[c for c in test if c != 'SeriousDlqin2yrs']+['SeriousDlqin2yrs']]

In [8]:
fixed_seed=12345
train80, test20 = train_test_split(fulltrain, test_size=0.2, RandomState=fixed_seed)
data_sets=[fulltrain,train80,test20,test]
filenames=[filename_fulltrain,filename_train,filename_valid,filename_test]

## Correcting the values in monthly income
Set NaN to 0 in the monthly income column

In [9]:
help(DataFrame.fillna)

Help on function fillna in module pandas.core.frame:

fillna(self, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None, **kwargs)
    Fill NA/NaN values using the specified method.
    
    Parameters
    ----------
    value : scalar, dict, Series, or DataFrame
        Value to use to fill holes (e.g. 0), alternately a
        dict/Series/DataFrame of values specifying which value to use for
        each index (for a Series) or column (for a DataFrame). (values not
        in the dict/Series/DataFrame will not be filled). This value cannot
        be a list.
    method : {'backfill', 'bfill', 'pad', 'ffill', None}, default None
        Method to use for filling holes in reindexed Series
        pad / ffill: propagate last valid observation forward to next valid
        backfill / bfill: use NEXT valid observation to fill gap
    axis : {0 or 'index', 1 or 'columns'}
    inplace : boolean, default False
        If True, fill in place. Note: this will modify any

In [10]:
for df in data_sets:
    df.loc[:,'MonthlyIncome'].fillna(0, inplace=True)
    
data_sets[0].head(10)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,1
2,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0,0
3,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0,0
4,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0,0
5,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0,0
6,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0,0
7,0.305682,57,0,5710.000000,0.0,8,0,3,0,0.0,0
8,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0,0
9,0.116951,27,0,46.000000,0.0,2,0,0,0,NaN,0
10,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0,0


## Cleaning Debt ratio
Set NA values to the mode from the training set

In [11]:
mode_debt_ratio=train80['DebtRatio'].mode

for df in data_sets:
    df.loc[:,'DebtRatio'].fillna(mode_debt_ratio, inplace=True)
    
data_sets[0].head(20)

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,1
2,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0,0
3,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0,0
4,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0,0
5,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0,0
6,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0,0
7,0.305682,57,0,5710.000000,0.0,8,0,3,0,0.0,0
8,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0,0
9,0.116951,27,0,46.000000,0.0,2,0,0,0,NaN,0
10,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0,0


## Creating a new column for monthly debt.

In [12]:
help(DataFrame.insert)

Help on function insert in module pandas.core.frame:

insert(self, loc, column, value, allow_duplicates=False)
    Insert column into DataFrame at specified location.
    
    Raises a ValueError if `column` is already contained in the DataFrame,
    unless `allow_duplicates` is set to True.
    
    Parameters
    ----------
    loc : int
        Insertion index. Must verify 0 <= loc <= len(columns)
    column : string, number, or hashable object
        label of the inserted column
    value : int, Series, or array-like
    allow_duplicates : bool, optional



In [13]:
for df in data_sets:
    df.insert(5,"MonthlyDebt",0)
data_sets[0].head(10)

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
1,0.766127,45,2,0.802982,9120.0,0,13,0,6,0,2.0,1
2,0.957151,40,0,0.121876,2600.0,0,4,0,0,0,1.0,0
3,0.658180,38,1,0.085113,3042.0,0,2,1,0,0,0.0,0
4,0.233810,30,0,0.036050,3300.0,0,5,0,0,0,0.0,0
5,0.907239,49,1,0.024926,63588.0,0,7,0,1,0,0.0,0
6,0.213179,74,0,0.375607,3500.0,0,3,0,1,0,1.0,0
7,0.305682,57,0,5710.000000,0.0,0,8,0,3,0,0.0,0
8,0.754464,39,0,0.209940,3500.0,0,8,0,0,0,0.0,0
9,0.116951,27,0,46.000000,0.0,0,2,0,0,0,NaN,0
10,0.189169,57,0,0.606291,23684.0,0,9,0,4,0,2.0,0


If there is no income set DebtRatio to the mean debt ratio.

In [14]:
for df in data_sets:
    df.loc[:,'MonthlyDebt']=df['DebtRatio']*df['MonthlyIncome']
    df.loc[df['MonthlyIncome'] == 0,'MonthlyDebt']=df.loc[df['MonthlyIncome'] == 0,'DebtRatio']
    #df.loc[df['MonthlyIncome'] == 0,'AbsoluteDebt']=df.loc[df['MonthlyIncome'] == 0,'DebtRatio'] #0.86222
    
    #df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=0 #0.85533
    df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=None #0.8610
    #df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=mode_debt_ratio #0.8517

data_sets[0][['MonthlyDebt','DebtRatio','MonthlyIncome']].head(10)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,MonthlyDebt,DebtRatio,MonthlyIncome
1,7323.197016,0.802982,9120.0
2,316.878123,0.121876,2600.0
3,258.914887,0.085113,3042.0
4,118.963951,0.036050,3300.0
5,1584.975094,0.024926,63588.0
6,1314.624392,0.375607,3500.0
7,5710.000000,NaN,0.0
8,734.790059,0.209940,3500.0
9,46.000000,NaN,0.0
10,14359.393699,0.606291,23684.0


## Creating a balanced Income feature

In [15]:
for df in data_sets:
    #df.insert(6,'BalancedIncome',0)
    df.insert(6,'BalancedIncome',None)

In [16]:
for df in data_sets:
    income_positive = df['MonthlyIncome'] > df['MonthlyDebt']
    df.loc[income_positive,'BalancedIncome']= df.loc[income_positive,'MonthlyIncome'] - df.loc[income_positive,'MonthlyDebt']

data_sets[0].head(20)

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,BalancedIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.8,13,0,6,0,2.0,1
2,0.957151,40,0,0.121876,2600.0,316.878123,2283.12,4,0,0,0,1.0,0
3,0.658180,38,1,0.085113,3042.0,258.914887,2783.09,2,1,0,0,0.0,0
4,0.233810,30,0,0.036050,3300.0,118.963951,3181.04,5,0,0,0,0.0,0
5,0.907239,49,1,0.024926,63588.0,1584.975094,62003,7,0,1,0,0.0,0
6,0.213179,74,0,0.375607,3500.0,1314.624392,2185.38,3,0,1,0,1.0,0
7,0.305682,57,0,NaN,0.0,5710.000000,None,8,0,3,0,0.0,0
8,0.754464,39,0,0.209940,3500.0,734.790059,2765.21,8,0,0,0,0.0,0
9,0.116951,27,0,NaN,0.0,46.000000,None,2,0,0,0,NaN,0
10,0.189169,57,0,0.606291,23684.0,14359.393699,9324.61,9,0,4,0,2.0,0


## Cleaning the number of dependents column
Set to 0 the number of dependents when not available.

In [17]:
for df in data_sets:
    df.loc[:,'NumberOfDependents'].fillna(0, inplace=True)
    df.loc[:,'NumberOfDependents']=to_numeric(df['NumberOfDependents'],downcast='integer')
    
data_sets[0][['NumberOfDependents']].head(10)

,NumberOfDependents
1,2
2,1
3,0
4,0
5,0
6,1
7,0
8,0
9,0
10,2


## Add a balanced income per household members feature

In [18]:
for df in data_sets:
    df.insert(7,'IncomePerHouseholdMember',0)

In [19]:
for df in data_sets:
    df.loc[:,'IncomePerHouseholdMember']= df['BalancedIncome'] / (df['NumberOfDependents']+1)
data_sets[0][['IncomePerHouseholdMember']].head(20)

,IncomePerHouseholdMember
1,598.934
2,1141.56
3,2783.09
4,3181.04
5,62003
6,1092.69
7,NaN
8,2765.21
9,NaN
10,3108.2


## Adding a "number of times late" categorical feature

In [20]:
for df in data_sets:
    df.insert(8,'LateCategory',0)

In [21]:
data_sets[0].loc[data_sets[0]['NumberOfTimes90DaysLate']==98,['LateCategory','NumberOfTimes90DaysLate']].head(20)

,LateCategory,NumberOfTimes90DaysLate
1734,0,98
2287,0,98
3885,0,98
4418,0,98
4706,0,98
5074,0,98
6281,0,98
7033,0,98
7118,0,98
7688,0,98


In [22]:
for df in data_sets:
    df.loc[:,'LateCategory']="L0"
    df.loc[df['NumberOfTimes90DaysLate'] == 98,'LateCategory']="L98"
    df.loc[df['NumberOfTimes90DaysLate'] == 96,'LateCategory']="L96"
    df.loc[(df['NumberOfTimes90DaysLate'] == 98) | (df['NumberOfTimes90DaysLate'] == 96),
              ['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']]=None
              
data_sets[0].loc[(data_sets[0]['LateCategory']=='L96') | (data_sets[0]['LateCategory']=='L98'),['LateCategory','NumberOfTimes90DaysLate']].head(20)

,LateCategory,NumberOfTimes90DaysLate
1734,L98,NaN
2287,L98,NaN
3885,L98,NaN
4418,L98,NaN
4706,L98,NaN
5074,L98,NaN
6281,L98,NaN
7033,L98,NaN
7118,L98,NaN
7688,L98,NaN


## Add a "Late score" feature

In [23]:
for df in data_sets:
    df.insert(0,'LateScore',0)

In [24]:
weight=[1,2,3]
#weight=[1,2,4]
for df in data_sets:
    df.loc[:,'LateScore']=weight[2]*df['NumberOfTimes90DaysLate']+weight[1]*df['NumberOfTime60-89DaysPastDueNotWorse']+weight[0]*df['NumberOfTime30-59DaysPastDueNotWorse']
    
data_sets[0][['LateScore']].head(10) 

,LateScore
1,2.0
2,0.0
3,4.0
4,0.0
5,1.0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0


## Remove the features we don't need

In [25]:
print(list(data_sets[0]))

['LateScore', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'BalancedIncome', 'IncomePerHouseholdMember', 'LateCategory', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents', 'SeriousDlqin2yrs']


In [26]:
for df in data_sets:
    df.drop('NumberOfTime30-59DaysPastDueNotWorse', axis=1, inplace=True)
    df.drop('NumberOfTime60-89DaysPastDueNotWorse', axis=1, inplace=True)
    df.drop('NumberOfTimes90DaysLate', axis=1, inplace=True)
    df.drop('NumberOfDependents', axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [27]:
print(list(data_sets[0]))

['LateScore', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'BalancedIncome', 'IncomePerHouseholdMember', 'LateCategory', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'SeriousDlqin2yrs']


## Save the data frames as csv files

In [28]:
for i in range(4):
    data_sets[i].to_csv(filenames[i],index_label='Id')